In [128]:
import gzip
import random
import scipy
import tensorflow as tf
from collections import defaultdict
import csv

In [129]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)
def readCSV(path):
    f = gzip.open(path, 'rt')
    c = csv.reader(f)
    header = next(c)
    for l in c:
        d = dict(zip(header,l))
        yield d['user_id'],d['recipe_id'],d['rating']

In [130]:
interactions = []
for l in readCSV("assignment1/trainInteractions.csv.gz"):
    interactions.append(l)

In [131]:
random.shuffle(interactions)
len(interactions)

500000

In [132]:
nTrain = int(len(interactions) * 0.9)
nTest = len(interactions) - nTrain
interactionsTrain = interactions[:nTrain]
interactionsTest = interactions[nTrain:]

In [133]:
itemsPerUser = defaultdict(list)
usersPerItem = defaultdict(list)
userIDs,itemIDs = {},{}
for u,i,r in interactions:
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in itemIDs: itemIDs[i] = len(itemIDs)
    itemsPerUser[u].append(i)
    usersPerItem[i].append(u)

In [134]:
mu = sum([int(r) for _,_,r in interactionsTrain]) / len(interactionsTrain)

In [135]:
optimizer = tf.keras.optimizers.Adam(0.1)

In [136]:
class LatentFactorModel(tf.keras.Model):
    def __init__(self, mu, K, lamb):
        super(LatentFactorModel, self).__init__()
        # Initialize to average
        self.alpha = tf.Variable(mu)
        # Initialize to small random values
        self.betaU = tf.Variable(tf.random.normal([len(userIDs)],stddev=0.001))
        self.betaI = tf.Variable(tf.random.normal([len(itemIDs)],stddev=0.001))
        self.gammaU = tf.Variable(tf.random.normal([len(userIDs),K],stddev=0.001))
        self.gammaI = tf.Variable(tf.random.normal([len(itemIDs),K],stddev=0.001))
        self.lamb = lamb

    # Prediction for a single instance (useful for evaluation)
    def predict(self, u, i):
        if (u == -5) and (i == -5):
            return self.alpha
        if u == -5:
            return self.alpha + self.betaI[i]
        if i == -5:
            return self.alpha + self.betaU[u]
        p = self.alpha + self.betaU[u] + self.betaI[i] +\
            tf.tensordot(self.gammaU[u], self.gammaI[i], 1)
        return p

    # Regularizer
    def reg(self):
        return self.lamb * (tf.reduce_sum(self.betaU**2) +\
                            tf.reduce_sum(self.betaI**2) +\
                            tf.reduce_sum(self.gammaU**2) +\
                            tf.reduce_sum(self.gammaI**2))
    
    # Prediction for a sample of instances
    def predictSample(self, sampleU, sampleI):
        u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
        i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
        beta_u = tf.nn.embedding_lookup(self.betaU, u)
        beta_i = tf.nn.embedding_lookup(self.betaI, i)
        gamma_u = tf.nn.embedding_lookup(self.gammaU, u)
        gamma_i = tf.nn.embedding_lookup(self.gammaI, i)
        pred = self.alpha + beta_u + beta_i +\
               tf.reduce_sum(tf.multiply(gamma_u, gamma_i), 1)
        return pred
    
    # Loss
    def call(self, sampleU, sampleI, sampleR):
        pred = self.predictSample(sampleU, sampleI)
        r = tf.convert_to_tensor(sampleR, dtype=tf.float32)
        return tf.nn.l2_loss(pred - r) / len(sampleR)

In [140]:
models = []
for K in [50]:
    for lambda_ in [0.0001]:
        models.append(LatentFactorModel(mu, K, lambda_))

In [141]:
def trainingStep(model, interactions):
    Nsamples = 50000
    with tf.GradientTape() as tape:
        sampleU, sampleI, sampleR = [], [], []
        for _ in range(Nsamples):
            u,i,r = random.choice(interactions)
            sampleU.append(userIDs[u])
            sampleI.append(itemIDs[i])
            sampleR.append(int(r))

        loss = model(sampleU,sampleI,sampleR)
        loss += model.reg()
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients((grad, var) for
                              (grad, var) in zip(gradients, model.trainable_variables)
                              if grad is not None)
    return loss.numpy()

In [142]:
for model in models:
    print('-----')
    for i in range(500):
        obj = trainingStep(model, interactions)
        if (i % 10 == 9): print("iteration " + str(i+1) + ", objective = " + str(obj))
#     predicted = []
#     labels = []
#     for u,i,r in interactionsTest:
#         uID = -5
#         iID = -5
#         if u in userIDs:
#             uID = userIDs[u]
#         if i in itemIDs:
#             iID = itemIDs[i]
#         predicted.append(model.predict(uID, iID).numpy())
#         labels.append(int(r))
#     print(MSE(predicted,labels))

-----
iteration 10, objective = 0.45854414
iteration 20, objective = 0.46706083
iteration 30, objective = 0.46305022
iteration 40, objective = 0.46089146
iteration 50, objective = 0.46045536
iteration 60, objective = 0.4376474
iteration 70, objective = 0.44939116
iteration 80, objective = 0.45474106
iteration 90, objective = 0.47012612
iteration 100, objective = 0.44837886
iteration 110, objective = 0.46306804
iteration 120, objective = 0.45422488
iteration 130, objective = 0.45469826
iteration 140, objective = 0.43750662
iteration 150, objective = 0.4530614
iteration 160, objective = 0.45123932
iteration 170, objective = 0.4516597
iteration 180, objective = 0.45717004
iteration 190, objective = 0.45294148
iteration 200, objective = 0.43435368
iteration 210, objective = 0.45474082
iteration 220, objective = 0.44722545
iteration 230, objective = 0.45162982
iteration 240, objective = 0.45476252
iteration 250, objective = 0.4579763
iteration 260, objective = 0.4425095
iteration 270, objec

In [13]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [80]:
MSE(predicted,labels)


0.827023822065572

In [143]:
predictions = open("predictions_Rated_tensorflow.txt", 'w')
for l in open("assignment1/stub_Rated.txt"):
    if l.startswith("user_id"):
        #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    uID = -5
    iID = -5
    if u in userIDs:
        uID = userIDs[u]
    if i in itemIDs:
        iID = itemIDs[i]
    num = models[0].predict(uID, iID).numpy()
    prediction = min(num,5)
    predictions.write(u + '-' + i + ',' + str(prediction) + '\n')

    

predictions.close()